In [96]:
import os
import numpy as np
import soundfile as sf
import librosa
import tensorflow as tf
import tensorflow_hub as hub

# Step 1: Load the VGGish model from TensorFlow Hub
vggish_model = hub.load('https://tfhub.dev/google/vggish/1')
print("VGGish model loaded successfully.")

# Freezing the VGGish model weights
vggish_model.trainable = False
print("VGGish model weights frozen.")

# Step 2: Define the input shape for raw waveforms (variable length audio)
input_waveform = tf.keras.Input(shape=(None,), dtype=tf.float32, name='waveform')
print("Input layer defined with variable length shape.")

def process_vggish(waveform):
    sample_rate = 16000
    chunk_size = 15360  # 0.96s of audio = 15360 samples at 16kHz
    
    # Pad the waveform to ensure it can be divided into 0.96s chunks
    num_samples = tf.shape(waveform)[-1]
    padding_needed = chunk_size - (num_samples % chunk_size)
    waveform_padded = tf.pad(waveform, [[0, padding_needed]])
    
    # Reshape the padded waveform to chunks of 0.96s (15360 samples)
    waveform_chunks = tf.reshape(waveform_padded, [-1, chunk_size])
    
    # Get the embeddings from VGGish
    embeddings = vggish_model(waveform_chunks)
    
    # Average the embeddings if there are multiple chunks
    embedding_mean = tf.reduce_mean(embeddings, axis=0)
    print(embedding_mean)
    return embedding_mean

# Step 4: Use a Lambda layer to incorporate the VGGish processing
vggish_output = tf.keras.layers.Lambda(process_vggish, output_shape=(128,))(input_waveform)
print("Input processed through VGGish.")

# Step 5: Add a classification layer
num_classes = 13  # Example number of classes
logits = tf.keras.layers.Dense(num_classes, activation='softmax', name='logits')(vggish_output)
print("Added classification layer.")

# Step 6: Create the Keras Model
model = tf.keras.Model(inputs=input_waveform, outputs=logits)
print("Keras model created successfully.")

# Ensure the new classification layer is trainable
for layer in model.layers:
    if layer.name == 'logits':
        layer.trainable = True
    else:
        layer.trainable = False
print("Only the classification layer is trainable.")

# Step 7: View the model summary
model.summary()

# Step 9: Load the mock .wav files and prepare a TensorFlow Dataset
def load_audio_file(file_path, sample_rate=16000):
    audio, sr = librosa.load(file_path, sr=sample_rate)
    return audio

def get_label_from_filename(filename):
    return int(filename.split('_')[1].split('.')[0])  # Assuming filename format "chord_X.wav"

def create_dataset_from_directory(directory, sample_rate=16000):
    audio_files = []
    labels = []
    
    for file_name in os.listdir(directory):
        if file_name.endswith('.wav'):
            file_path = os.path.join(directory, file_name)
            audio = load_audio_file(file_path, sample_rate)
            label = get_label_from_filename(file_name)
            
            audio_files.append(audio)
            labels.append(label)
    
    # Convert to TensorFlow Dataset
    dataset = tf.data.Dataset.from_tensor_slices((np.array(audio_files), np.array(labels)))
    return dataset.batch(2)

# Step 10: Train the model on mock data
train_dataset = create_dataset_from_directory(r'../data/raw/')

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

print("Training model...")
model.fit(train_dataset, epochs=5)
print("Model trained successfully.")

# Step 11: Save the model
save_path = './saved_model/vggish_classifier'
model.save(save_path)
print(f"Model saved successfully to {save_path}")


VGGish model loaded successfully.
VGGish model weights frozen.
Input layer defined with variable length shape.
Input processed through VGGish.
Added classification layer.
Keras model created successfully.
Only the classification layer is trainable.


Model: "functional_29"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                      ┃ Output Shape             ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ waveform (InputLayer)             │ (None, None)             │             0 │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ lambda_30 (Lambda)                │ (None, 128)              │             0 │
├───────────────────────────────────┼──────────────────────────┼───────────────┤
│ logits (Dense)                    │ (None, 13)               │         1,677 │
└───────────────────────────────────┴──────────────────────────┴───────────────┘

 Total params: 1,677 (6.55 KB)

 Trainable params: 1,677 (6.55 KB)

 Non-trainable params: 0 (0.00 B)

IndexError: list index out of range